**Part 1: Hands-On #2 (0 pts)**

Import and view data.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

TweetUrl='https://github.com/aasiaeet/cse5522data/raw/master/db3_final_clean.csv'
tweet_dataframe=pd.read_csv(TweetUrl)

tweet_dataframe.head()

weight                                              tweet  label
0  1.0000          it is very cold out want it to be warmer      -1
1  0.7698  dammmmmmm its pretty cold this morning   burr lol     -1
2  0.6146  why does halsey have to be so far away think m...     -1
3  0.9356         dammit stop being so cold so can work out      -1
4  1.0000                               its too freakin cold     -1

Ignoring repeated words, put every word into a dictionary ```(word, id)```



In [ ]:
wordDict = {}
idCounter = 0
for index, weight, tweet, label in tweet_dataframe.itertuples():
  for word in tweet.split(" "):
    if word not in wordDict:
      wordDict[word] = idCounter
      idCounter += 1


Create a ```[num_tweets x num_unique_words]``` document-word matrix and fill it with zeroes.

In [ ]:
x = np.zeros((tweet_dataframe.shape[0], idCounter),dtype='float')

```y``` is the array of tweet labels.

In [ ]:
y = np.array(tweet_dataframe.iloc[:,2])

Each row represents a tweet and each column represents a unique word. They are all initialzed as zeroes. Go through the tweets and mark their occurence.

In [ ]:
for i, w, tweet, l in tweet_dataframe.itertuples():
  for word in tweet.split(" "):
    x[i, wordDict[word]] = 1

Check our work and expect: 

```array([10.,  9., 17.,  9.,  4.])```

In [ ]:
np.sum(x[0:5, ], axis = 1)

array([10.,  9., 17.,  9.,  4.])

Compute totals and their percentages.

In [ ]:
numNeg = np.sum(y<0)
numPos = np.sum(y>=0) #len(y) - numNeg
probNeg = numNeg / (numNeg + numPos)
probPos = 1 - probNeg

Do an 80/20 train/test split.


In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2)

Learn the world probabilities.

In [ ]:
def compute_distros(x, y):
  probWordGivenPositive=np.mean(x[y>=0,:],axis=0)     # P(word|Sentiment = +ive)
  probWordGivenNegative=np.mean(x[y<0,:],axis=0)      # P(word|Sentiment = -ive)
  priorPositive = np.mean(y>=0)                       # P(Sentiment = +ive)
  priorNegative = 1 - priorPositive                   # P(Sentiment = -ive)

  return probWordGivenPositive, probWordGivenNegative, priorPositive, priorNegative


probWordGivenPositive, probWordGivenNegative, priorPositive, priorNegative = compute_distros(xTrain,yTrain)

Note that you only needed to compute $P(word = 1| +)$ or $P(word = 1| -)$ and the probabilities of the word being absent from a tweet is just 1 minus those probabilities. 

We will also want to compute $log P(word = 1 | +)$, $log P(word = 0 | +)$, $log P(word = 1 | -)$ and $log P(word = 0 | -)$.  Also we should compute the log priors.  Let's do so now.


In [ ]:
def compute_logdistros(distros, min_prob):
  if True:
    #Assume missing words are simply very rare
    #So, assign minimum probability to very small elements (e.g. 0 elements)
    distros=np.where(distros>=min_prob,distros,min_prob)
    #Also need to consider minimum probability for "not" distribution
    distros=np.where(distros<=(1-min_prob),distros,1-min_prob)

    return np.log(distros), np.log(1-distros)
  else:
    #Ignore missing words (assume they have P==1, i.e. force log 0 to 0)
    return np.log(np.where(distros>0,distros,1)), np.log(np.where(distros<1,1-distros,1))

min_prob = 1/yTrain.shape[0] #Assume very rare words only appeared once
logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive = compute_logdistros(probWordGivenPositive,min_prob)
logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative = compute_logdistros(probWordGivenNegative,min_prob)
logPriorPositive, logPriorNegative = compute_logdistros(priorPositive,min_prob)

**Math of NB** Here we provide the derivation of NB when we want to classify the $i$th tweet $\textbf{x}^{(i)}$ and the size of dictionary is $p$, i.e., each tweet is a binary vector of size $p$ as $\textbf{x}^{(i)} = (x_1^{(i)},\dots, x_p^{(i)})$. 

Note that we computed $P(x_j^{(i)} = 1|+)$ and $P(x_j^{(i)} = 1|-)$ in above code from `xTrain` and now want to classify `xTest` samples.

**Classification Rule:** For each tweet $i$ NB classifier assigns label + if $P(+|\textbf{x}^{(i)}) > P(-|\textbf{x}^{(i)})$ and negative otherwise. 

These posterior probabilities can be computed using prior probabilities (that we got from `xTrain`) and Bayes rule as follows: 

\begin{align}
P(+|\textbf{x}^{(i)}) &= \alpha P(\{\textbf{x}^{(i)}\}_{i=1}^n | +)P(+) 
\\
(\text{NB Assumption}) &= \alpha P(+) \prod_{j=1}^p P(x_j^{(i)}|+)
\end{align}

For computational convinence (preventing underflow while dealing with small numbers) we work with the $\log$ of probabilities:

\begin{align} 
\log(P(+|\textbf{x}^{(i)})) &\propto \log P(+) + \sum_{j=1}^p \log P(x_j^{(i)}|+) 
\\
\log(P(-|\textbf{x}^{(i)})) &\propto \log P(-) + \sum_{j=1}^p \log P(x_j^{(i)}|-) 
\end{align} 

Finally we can compute the confidence of our prediction as the log of the ratio of posteriors: 
$\log(\frac{P(\text{predicted label}|\textbf{x}^{(i)})}{P(\text{the other label}|\textbf{x}^{(i)})})$

Below, we write a function that takes a row of `xTest` and output a label for it based on NB classification rule. 

In [ ]:
import math

# classifyNB: 
#   words - vector of words of the tweet (binary vector)
#   logProbWordPresentGivenPositive - log P(x_j = 1|+)
#   logProbWordAbsentGivenPositive  - log P(x_j = 0|+)
#   logProbWordPresentGivenNegative - log P(x_j = 1|-)
#   logProbWordAbsentGivenNegative  - log P(x_j = 0|-)
#   logPriorPositive - log P(+)
#   logPriorNegative - log P(-)
#   returns (label of x according to the NB classification rule, confidence about the label)

def classifyNB(words,logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
               logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
               logPriorPositive, logPriorNegative):

  p_plus = logPriorPositive
  p_minus = logPriorNegative

  for i in range(len(words)):
    if words[i] == 0:
      p_plus += logProbWordAbsentGivenPositive[i]
      p_minus += logProbWordAbsentGivenNegative[i]
    else:
      p_plus += logProbWordPresentGivenPositive[i]
      p_minus += logProbWordPresentGivenNegative[i]

  sentiment = 1
  predicted_label = p_plus
  other_label = p_minus

  if p_minus > p_plus:
    sentiment = -1
    predicted_label = p_minus
    other_label = p_plus
  
  return sentiment, abs(math.log(predicted_label/other_label))

  
  
print(classifyNB(xTest[700, ], logProbWordPresentGivenPositive,logProbWordAbsentGivenPositive,
                               logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative,
                               logPriorPositive, logPriorNegative))

(1, 0.08196431113886807)


Compute the output of classifyNB for all test data and output the average error.


In [ ]:
# testNB: Classify all xTest
#   xTest - test data features
#   yTest - true label of test data
#   logProbWordPresentGivenPositive - log P(x_j = 1|+)
#   logProbWordAbsentGivenPositive  - log P(x_j = 0|+)
#   logProbWordPresentGivenNegative - log P(x_j = 1|-)
#   logProbWordAbsentGivenNegative  - log P(x_j = 0|-)
#   logPriorPositive - log P(+)
#   logPriorNegative - log P(-)
#   returns Average test error
def testNB(xTest, yTest, 
           logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
           logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
           logPriorPositive, logPriorNegative, classification_function):
  
  numCorrect = 0

  for i, words in enumerate(xTest):
    sent, conf = classification_function(words, logProbWordPresentGivenPositive,logProbWordAbsentGivenPositive,
                               logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative,
                               logPriorPositive, logPriorNegative)
    
    if sent == yTest[i]:
      numCorrect +=1

    avgErr = numCorrect / (i + 1)
  
  #print("Average error of NB is", avgErr)
  return avgErr

testNB(xTest, yTest, 
       logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
       logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
       logPriorPositive, logPriorNegative, classifyNB)



0.8297297297297297

Do this with 10 train/test splits

In [ ]:
results1 = []

for _ in range(10):

  xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2)

  probWordGivenPositive, probWordGivenNegative, priorPositive, priorNegative = compute_distros(xTrain,yTrain)
  min_prob = 1/yTrain.shape[0] 
  logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive = compute_logdistros(probWordGivenPositive,min_prob)
  logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative = compute_logdistros(probWordGivenNegative,min_prob)
  logPriorPositive, logPriorNegative = compute_logdistros(priorPositive,min_prob)

  results1.append(testNB(xTest, yTest, 
       logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
       logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
       logPriorPositive, logPriorNegative, classifyNB))
  
test1 = np.mean(results1)  
print(f"Model 1: {(test1*100.0):.3f}%")


Model 1: 83.203%



Compare with sklearn.

In [ ]:
from sklearn.naive_bayes import MultinomialNB

skresults = []

for _ in range(10):
  xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2)

  mnb = MultinomialNB()
  mnb.fit(xTrain, yTrain)
  accuracy = mnb.score(xTest, yTest)
  skresults.append(accuracy)

testsk = np.mean(skresults)
print(f"sklearn: {testsk*100.0:.3f}%")


sklearn: 82.959%


---
---

**Part 2: An Alternate Model (50 pts)**

Modify your code to *only* incorporate the probability of words present in the tweet $P\left(x^i_j=1 | +\right)$ (thus ignoring absent words).

Compare this to the original approach in Part 1. Follow reasonable experimental procedure and write up an explanation of the results you find.


In [ ]:
def classifyNB_alt(words,logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
               logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
               logPriorPositive, logPriorNegative):

  p_plus = logPriorPositive
  p_minus = logPriorNegative

  for i in range(len(words)):
    if words[i] > 0:
      p_plus += logProbWordPresentGivenPositive[i]
      p_minus += logProbWordPresentGivenNegative[i]

  sentiment = 1
  predicted_label = p_plus
  other_label = p_minus

  if p_minus > p_plus:
    sentiment = -1
    predicted_label = p_minus
    other_label = p_plus
  
  return sentiment, abs(math.log(predicted_label/other_label))


results2 = []

for _ in range(10):

  xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2)

  probWordGivenPositive, probWordGivenNegative, priorPositive, priorNegative = compute_distros(xTrain,yTrain)
  min_prob = 1/yTrain.shape[0] 
  logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive = compute_logdistros(probWordGivenPositive,min_prob)
  logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative = compute_logdistros(probWordGivenNegative,min_prob)
  logPriorPositive, logPriorNegative = compute_logdistros(priorPositive,min_prob)

  results2.append(testNB(xTest, yTest, 
       logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
       logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
       logPriorPositive, logPriorNegative, classifyNB_alt))

test2 = np.mean(results2)
print(f"Model 2: {test2*100.0:.3f}%")

Model 2: 82.892%


First I want to know how much data this second model excluded.

In [ ]:
average_remaining_data = [(np.count_nonzero(row)/len(row))*100.0 for row in x]
total_remaining_data = np.count_nonzero(x)
overall_prop_remaining = x.size


print(f"The document word matrix has {x.size} elements, {x.size - np.count_nonzero(x)} of which were not considered.")

The document word matrix has 22141333 elements, 22091805 of which were not considered.


For the second model we did not use the probability of abscence in our prediction. I expected this modification to improve the model because probability of absence may be a source of noise in this model. Given that our training set consisted of short tweets aside a massive dictionary, most words are likely to be absent, regardless of sentiment. Trying to model off of that would introduce noise. Given that removing a massive amount of data caused a minor improvement, I think this is true.

---
---
**Part 3: An Additional Experiment (50 pts)**

**3.1: Option 1: Removing stop words**

Investigate the effect of removing the 25, 50, 100, and 200 most frequent words from the calculation.


Remember, you need to *compare* your chosen option with your previous work. Just writing the code is not sufficient. Follow reasonable experimental procedure and write up a discussion of your results and why you think they turned out that way.

In [ ]:
# Option 3.1
num_removed = [25, 50, 100, 200]
num_to_remove = [25, 25, 50, 100]

# to get occurence count of each word we sum the columns of X
# a bad way to do this is with dot product
sums = np.dot(x.T, np.ones(x.shape[0]))
# now we want to sort by that vector:
x = x[:, np.argsort(sums)]


results3 = []

for idx, remove in enumerate(num_to_remove):

  x = x[:, :-remove]

  xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2)

  probWordGivenPositive, probWordGivenNegative, priorPositive, priorNegative = compute_distros(xTrain,yTrain)
  min_prob = 1/yTrain.shape[0] 
  logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive = compute_logdistros(probWordGivenPositive,min_prob)
  logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative = compute_logdistros(probWordGivenNegative,min_prob)
  logPriorPositive, logPriorNegative = compute_logdistros(priorPositive,min_prob)

  results3.append(testNB(xTest, yTest, 
       logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
       logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
       logPriorPositive, logPriorNegative, classifyNB_alt))

for i, result in enumerate(results3):
  print(f" {num_removed[i]} words removed: {result*100:.3f}%")

 25 words removed: 81.486%
 50 words removed: 77.027%
 100 words removed: 75.541%
 200 words removed: 67.703%


Removal of stop words can be useful for some tasks but not so much for others. While it does greatly reduce training time and increase the concentration of unique information, it can negatively effect some NLP tasks such as sentiment analysis. Words like "not", "but", and "good" are very common and can completely change the meaning of a sentence. Given the sentence, "I did not enjoy learning ruby on rails", an agressive removal of stop words can result in something like "did enjoy learning ruby rails" which is the absolute opposite sentiment of the intended. Perhaps an approach that determines which stop words should be retained and which should be culled can give is the best of both worlds.